In [1]:
import json

with open('data/clean/stations.json', encoding='utf-8') as file:
    file_contents = file.read()

data = json.loads(file_contents)
graph = []
for i in data:
    for j in i['connections']:
        graph.append([i['name'], j['name'], int(j['cost']), 1, j['fromline']])

node_lines = dict()
for i in data:
    node_lines[i['name']] = i['line']
temp = []
temp1 = []
totalcost = 0
for i in graph:
    temp.append(i[0])
    temp1.append(i[1])
nodes = set(temp).union(set(temp1))


In [2]:
def UCS(graph, costs, open, closed, cur_node, totalcosts, line):
    if cur_node in open:
        open.remove(cur_node)
    closed.add(cur_node)
    for i in graph:
        # cal the distance and the heurisitc of current node
        # distance(start to current node) = costs[i[0]][0]+i[2]
        # heurisitc(current node) = i[3]
        if (i[0] == cur_node and costs[i[0]]+i[2] < costs[i[1]] and i[1] not in closed):
            open.add(i[1])
            walktime = 0
            
            print('cur node '+cur_node)
            print('next node '+i[1])
            cur_line = line[i[0]].copy()
            print(i[4])
            print(cur_line)
            if len(cur_line) == 0:
                cur_line.append(i[4])
            elif i[4] not in cur_line:
                cur_line.append(i[4])
                walktime = 3
            print(cur_line)
            print(line)
            print('last line', end=' ')
            print(line[i[0]])
            print('cur line', end=' ')
            print(line[i[1]])
            line[i[1]] = cur_line
            costs[i[1]] = costs[i[0]]+i[2]+walktime
            print('cost: ', costs[i[1]])
            # store the heurisitc of current node
            # costs[i[1]][1] = i[3]
            path[i[1]] = path[i[0]] + ' -> ' + i[1]
            # totalcosts[i[1]] = sum(costs[i[1]])
            totalcosts[i[1]] = costs[i[1]]

    costs[cur_node] = 999999
    small = min(costs, key=costs.get)
    if small not in closed:
        UCS(graph, costs, open, closed, small, totalcosts, line)


costs = dict()
path = dict()
totalcosts = dict()
line = dict()
for i in nodes:
    # costs array contain the distance and the heurisitc of current node
    costs[i] = 999999
    path[i] = ' '
    totalcosts[i] = 999999
    line[i] = []

open = set()
closed = set()
start_node = input("Enter the Start Node: ")
open.add(start_node)
path[start_node] = start_node
costs[start_node] = 0
totalcosts[start_node] = 0
UCS(graph, costs, open, closed, start_node, totalcosts, line)
goal_node = input("Enter the Goal Node: ")
print("Suggested Route:")
print(path[goal_node])
print("Interchange times: ", len(line[goal_node])-1)
print("Suggested Line: ", line[goal_node])
print("Least cost is: ",totalcosts[goal_node])

print("All the nodes and their costs:")
print(totalcosts)


cur node Tsuen Wan
next node Tai Wo Hau
Tsuen Wan Line
[]
['Tsuen Wan Line']
{'Tin Hau': [], 'Ma On Shan': [], 'Yau Tong': [], 'Ngau Tau Kok': [], 'Sai Ying Pun': [], 'Austin': [], 'Hong Kong': [], 'Tai Wai': [], 'Prince Edward': [], 'Fo Tan': [], 'Chai Wan': [], 'Kai Tak': [], 'Mei Foo': [], 'North Point': [], 'Lam Tin': [], 'Shau Kei Wan': [], 'Kwai Fong': [], 'Tai Shui Hang': [], 'Kwai Hing': [], 'Tsuen Wan West': [], 'East Tsim Sha Tsui': [], 'Fanling': [], 'Admiralty': [], 'Wan Chai': [], 'Tai Wo Hau': [], 'Mong Kok': [], 'Kwun Tong': [], 'Hin Keng': [], 'Kam Sheung Road': [], 'Po Lam': [], 'Sheung Wan': [], 'Choi Hung': [], 'Kowloon Bay': [], 'Olympic': [], 'Tiu Keng Leng': [], 'Jordan': [], 'City One': [], 'Siu Hong': [], 'Sheung Shui': [], 'Causeway Bay': [], 'Ho Man Tin': [], 'Lei Tung': [], 'Yuen Long': [], 'Yau Ma Tei': [], 'Central': [], 'Tuen Mun': [], 'Kowloon Tong': [], 'Tsim Sha Tsui': [], 'Kowloon': [], 'Che Kung Temple': [], 'Lok Fu': [], 'Long Ping': [], 'Tung Chung'